In [ ]:
import smartsheet # need version 2.105.0
import PyPDF2
import pandas as pd
import re
import requests
import pyodbc
import smartsheet
import re
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, inspect
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None


api_key = 'smartsheet_key'
sheet_id = 'smartsheet_id'

# Load the required Jira tables to get the records for SLAs that have not completed processing

In [ ]:
# Database connection settings
server = 'server_path'
database = 'database_name'
username = 'username'
password = 'password'
view_name_bic_issues = 'dbo.tablename'
view_name_bic_top_level_issues = 'dbo.tablename'

# Establishing the database connection
conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
conn = pyodbc.connect(conn_str)

# SQL query to fetch the view data
query1 = f'SELECT * FROM {view_name_bic_issues}'
query2 = f'SELECT * FROM {view_name_bic_top_level_issues}'

# Executing the query and fetching the data
bic_issues = pd.read_sql_query(query1, conn)
top_level_issues = pd.read_sql_query(query2, conn)

# Closing the database connection
conn.close()



# Target records with SLA in the title

In [ ]:
# filter project keys to RDS
rds = bic_issues[bic_issues.Project_Key == 'RDS']

def check_for_sla(text):
    if 'SLA' in text:
        return 1
    else:
        return 0

# add a column that identifies if the 'Issue_Summary' field contains the text SLA and returns a 1 if SLA is in the text
rds['contains_sla'] = rds.apply(lambda x: check_for_sla(x.Issue_Summary), axis=1)

# filter to only SLA records 
sla_records = rds[rds.contains_sla == 1]

# filter sla records based on the date which the records should only be newly requested
# as prior records have been left open
date_from = pd.to_datetime('05/18/2023')
sla_records['Created_Date'] = pd.to_datetime(sla_records['Created_Date'])
sla_records = sla_records[sla_records['Created_Date'] >= date_from] #### pulled for testing 
#Issue_Ticket_Number = [2191, 2606, 2611]
#sla_records = sla_records[sla_records.Issue_Ticket_Number.isin(Issue_Ticket_Number)]
len(sla_records)

# Merge required fields from the parent ticket and update field titles

In [ ]:
# merge in required fields from the parent ticket
top_slice = top_level_issues[['Issue_ID',
                              'Issue_Status',
                              'Issue_Ticket_Number',
                              'Project_Key',
                              'Research_Info__Project_Title',
                              'Research_Info__PI_Email',
                              'Research_Info__PI_Name']]
top_slice.rename(columns={'Issue_ID':'Parent_Issue_ID',
                          'Issue_Ticket_Number':'Issue_Ticket_Number_parent', 
                          'Issue_Status':'Issue_Status_parent',
                          'Project_Key':'Project_Key_parent',
                          'Research_Info__Project_Title':'Research_Info__Project_Title_parent',
                          'Research_Info__PI_Email':'Research_Info__PI_Email_parent',
                          'Research_Info__PI_Name':'Research_Info__PI_Name_parent'}, inplace=True)


top_slice['Project_Key_parent_full'] = top_slice.apply(lambda x: x.Project_Key_parent +"-"+ str(x.Issue_Ticket_Number_parent), axis=1) 

sla_merged = pd.merge(sla_records, top_slice, how='left', on='Parent_Issue_ID')

sla_merged['Project_Key_child_full'] = sla_merged.apply(lambda x: x.Project_Key +"-"+ str(x.Issue_Ticket_Number), axis=1) 

# Parse the text data in the SLA Jira ticket to extract the required SLA features to build the SLA and apply "Jira Data Missing" via QC methods if there are missing values in the Jira ticket

In [ ]:

def parse_support_data(text):
    '''
    scope: parses the required data from the SLA subtask Issue_Description that has been formated to collect the scope and other
            data required for the SLA downstream processing
    '''
    # Initialize the dictionary with default values
    support_data = {
        "scope": "Jira Data Missing",
        "start": "Jira Data Missing",
        "end": "Jira Data Missing",
        "hours": 0,
        "r1": 0,
        "r2": 0
    }
    
    if text is not None:
        # Use regular expressions to extract data
        scope_match = re.search(r'^(.*?)###', text, re.DOTALL | re.IGNORECASE)
        if scope_match:
            support_data["scope"] = scope_match.group(1).strip()

        start_match = re.search(r'Start:\s*(\d{2}/\d{2}/\d{4})', text)
        if start_match:
            support_data["start"] = start_match.group(1)

        end_match = re.search(r'End:\s*(\d{2}/\d{2}/\d{4})', text)
        if end_match:
            support_data["end"] = end_match.group(1)

        hours_match = re.search(r'Total Hours:\s*(\d+)', text)
        if hours_match:
            support_data["hours"] = int(hours_match.group(1))

        r1_match = re.search(r'Additional Resource 1:\s*(.*?)\s*(Additional Resource|$)', text, re.DOTALL)
        if r1_match:
            r1_text = r1_match.group(1).strip()
            support_data["r1"] = r1_text if r1_text else 0

        r2_match = re.search(r'Additional Resource 2:\s*(.*?)\s*(Additional Resource|$)', text, re.DOTALL)
        if r2_match:
            r2_text = r2_match.group(1).strip()
            support_data["r2"] = r2_text if r2_text else 0

    return support_data


def sla_state_qc(row):
    '''
    scope: quality control check to determine if a manaul review is needed for required fields that
            are being derived from the Jira data and ETL to Smartsheets
    
    '''
    if row['estimated_project_start_date'] == 'Jira Data Missing':
        results = 'Jira Data Missing'
        
    elif row['estimated_project_end_date'] == 'Jira Data Missing':
        results = 'Jira Data Missing'
        
    elif row['total_estimated_hours'] == 0:
        results = 'Jira Data Missing'
        
    elif row['scope'] == '':
        results = 'Jira Data Missing'
        
    else:
        results = 'Format Review Required'
    return results



# add the project scope data and other supporting data needed for downstream SLA processing
sla_merged['scope'] = sla_merged.apply(lambda x: parse_support_data(x.Issue_Description)['scope'],axis=1)
sla_merged['estimated_project_start_date'] = sla_merged.apply(lambda x: parse_support_data(x.Issue_Description)['start'],axis=1)
sla_merged['estimated_project_end_date'] = sla_merged.apply(lambda x: parse_support_data(x.Issue_Description)['end'],axis=1)
sla_merged['total_estimated_hours'] = sla_merged.apply(lambda x: parse_support_data(x.Issue_Description)['hours'],axis=1)
sla_merged['additional_resource_1'] = sla_merged.apply(lambda x: parse_support_data(x.Issue_Description)['r1'],axis=1)
sla_merged['additional_resource_2'] = sla_merged.apply(lambda x: parse_support_data(x.Issue_Description)['r2'],axis=1)



# Replace "None" with empty string ("") and remove records that have "Jira Data Missing" from the scope
sla_merged = sla_merged.fillna('Jira Data Missing')
sla_merged = sla_merged[sla_merged.scope != 'Jira Data Missing']
sla_merged['SLA_state'] = sla_merged.apply(sla_state_qc, axis=1)   
len(sla_merged)

# Scope down the record set and update field titles

In [ ]:
# scope down the dataframe to only the required fields then preprocess so the field titles match what is in smartsheets 
scoped_records = sla_merged[['SLA_state',
                             'Project_Key_parent_full',
                             'Project_Key_child_full',
                             'Research_Info__Project_Title_parent',
                             'scope',
                             'estimated_project_start_date',
                             'estimated_project_end_date',
                             'Research_Info__PI_Name_parent',
                             'Research_Info__PI_Email_parent',
                             'total_estimated_hours',
                             'additional_resource_1',
                             'additional_resource_2']]


scoped_records.rename(columns={'Project_Key_parent_full':'jira_ticket_number',
                               'Project_Key_child_full':'jira_subtask_ticket_number',
                               'Research_Info__Project_Title_parent':'project_title',
                               'Research_Info__PI_Name_parent':'pi_name',
                               'Research_Info__PI_Email_parent':'pi_email'}, inplace=True)



# Create additional SLA required calculated fields

In [ ]:
# Convert specific columns to float data type
columns_to_convert = ['total_estimated_hours', 'additional_resource_1', 'additional_resource_2']
scoped_records[columns_to_convert] = scoped_records[columns_to_convert].astype(float)

# add in the other fields within the smartsheet and derive values
def total_hourly_cost(hours, gratis, rate):
    #
    result = (hours-gratis)*rate
    if result < 0:
        output = 0
    else:
        output = result
    return output 

def total_estimated_cost(hour_cost, r1, r2):
    
    result = hour_cost+ r1 + r2
    if result < 0:
        output = 0
    else:
        output = result
    return output 


scoped_records['gratis_hours'] = 8
scoped_records['hourly_rate'] = 60
scoped_records['total_hourly_cost'] = scoped_records.apply(lambda x: total_hourly_cost(x.total_estimated_hours, x.gratis_hours, x.hourly_rate ), axis=1) 
scoped_records['total_estimated_cost'] = scoped_records.apply(lambda x: total_estimated_cost(x.total_hourly_cost,x.additional_resource_1,x.additional_resource_2), axis=1)


# Load the current smartsheet record and use the "jira_ticket_number" field to create a unique list of the tickets that have already been posted and remove these from the scoped record set

In [ ]:


def extract_and_load_to_dataframe(api_token, sheet_id):
    # Initialize the Smartsheet client
    ss_client = smartsheet.Smartsheet(api_token)

    try:
        # Get the sheet information to access column details
        sheet = ss_client.Sheets.get_sheet(sheet_id)
        columns = sheet.columns

        # Extract column headers from the sheet
        column_headers = [column.title for column in columns]

        # Load the entire sheet
        sheet = ss_client.Sheets.get_sheet(sheet_id, page_size=10000)
        rows = sheet.rows

        # Create a list to hold row data
        data = []

        # Extract data from each row and add to the list
        for row in rows:
            row_data = []
            for cell in row.cells:
                row_data.append(cell.display_value)
            data.append(row_data)

        # Convert the list to a DataFrame with column headers
        df = pd.DataFrame(data, columns=column_headers)

        return df

    except Exception as e:
        print(f"Error: {e}")
        return None
current_smartsheet_df = extract_and_load_to_dataframe(api_key, sheet_id)
jira_ticket_list = list(set(list(current_smartsheet_df.jira_ticket_number)))

scoped_records = scoped_records[~scoped_records.jira_ticket_number.isin(jira_ticket_list)]


# Get the current listing of field names and unique IDs to match the field name from the scoped data to smartsheet 

In [ ]:
def get_column_name_to_id(sheet_id, api_key):
    # Initialize the Smartsheet client
    smartsheet_client = smartsheet.Smartsheet(api_key)

    try:
        # Load the specified sheet
        sheet = smartsheet_client.Sheets.get_sheet(sheet_id)

        # Initialize an empty dictionary to store column name to column ID mapping
        column_name_to_id = {}

        # Iterate through columns in the sheet
        for column in sheet.columns:
            # Store the column name (title) as the key and column ID as the value
            column_name_to_id[column.title] = column.id

        return column_name_to_id

    except Exception as e:
        print(f"Error: {e}")
        return None
    
# Call the function to get the column name to column ID mapping
result = get_column_name_to_id(sheet_id, api_key)

# If scope records exist then load the data into Smartsheets

In [ ]:

if len(scoped_records) >0:
    # Initialize the Smartsheet client
    smartsheet_client = smartsheet.Smartsheet(api_key)

    # Convert all columns to string data type
    new_records = scoped_records.astype(str)

    new_records_columns = list(set(list(scoped_records)))
    df_len = len(new_records)
    counter = 0
    # use a counter to iterate over the rows within the dataframe to load the data from each row into the smartsheet format
    while counter < df_len:
        # Specify cell values for one row
        row_a = smartsheet.models.Row()
        row_a.to_top = True
        # iterate over the field names in the scoped dataset and use these field names to call the 
        # key from the result dictionary that contains the smartsheet field ID associated with the key/title
        # and use this field ID to build the record data neede for smartsheet to post the row
        for column_name in new_records_columns:
            row_a.cells.append({
                'column_id': int(result[column_name]), 
                'value': new_records.iloc[counter][column_name]
            })
        # Add rows to sheet
        response = smartsheet_client.Sheets.add_rows(
            sheet_id=sheet_id,  
            list_of_rows=[row_a],
        )

        # Check for success or handle errors as needed
        if response.message == 'SUCCESS':
            print('Rows added successfully')
            counter = counter + 1
        else:
            print(f'Error: {response.message}')
            counter = counter + 1
else:
    print('No Records to Process')